In [344]:
# import Panda dependency
import pandas as pd

In [345]:
# filepath as strings, read as DFs
sch_file = "resources\schools_complete.csv"
stu_file = "resources\students_complete.csv"
sch_df = pd.read_csv(sch_file)
stu_df = pd.read_csv(stu_file)

sch_df

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [346]:
# clean student names using list, looping through, and using "replace" to change target string into desired value ("") -- blank in this case
presuffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

for psfix in presuffixes:
    stu_df["student_name"] = stu_df["student_name"].str.replace(psfix, "")

C:\Users\nabil\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [347]:
# merge both dfs for complete data set get basic metrics
all_data = pd.merge(stu_df, sch_df, on=["school_name", "school_name"])

student_count = all_data["Student ID"].count()
school_count = len(all_data["school_name"].unique())
total_budget = sch_df["budget"].sum()
mean_read = all_data["reading_score"].mean()
mean_math = all_data["math_score"].mean()

#pass or fail into new DFs, do counts, and percentages
pass_math = all_data[all_data["math_score"] > 69]
pass_read = all_data[all_data["reading_score"] > 69]
pass_both = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]

math_pass_count = pass_math["student_name"].count()
read_pass_count = pass_read["student_name"].count()
pass_both_count = pass_both["student_name"].count()

math_perc = math_pass_count / float(student_count) * 100
read_perc = read_pass_count / float(student_count) * 100
both_perc = pass_both_count / float(student_count) * 100

In [348]:
#REQUIREMENT #1

#create district summary df
dist_sum_df = pd.DataFrame(
    [{"Total Schools" : school_count,
    "Total Students" : student_count,
    "Total Budget" : total_budget,
    "Ave Math Score" : mean_math,
    "Ave Read Score" : mean_read,
    "% Pass Math" : math_perc,
    "% Pass Read" : read_perc,
    "% Overall Pass" : both_perc}])

#format each column appropriately
dist_sum_df["Total Students"] = dist_sum_df["Total Students"].map("{:,}".format)
dist_sum_df["Ave Math Score"] = dist_sum_df["Ave Math Score"].map("{:.1f}".format)
dist_sum_df["Ave Read Score"] = dist_sum_df["Ave Read Score"].map("{:.1f}".format)
dist_sum_df["% Pass Math"] = dist_sum_df["% Pass Math"].map("{:.0f}".format)
dist_sum_df["% Pass Read"] = dist_sum_df["% Pass Read"].map("{:.0f}".format)
dist_sum_df["% Overall Pass"] = dist_sum_df["% Overall Pass"].map("{:.0f}".format)
dist_sum_df["Total Budget"] = dist_sum_df["Total Budget"].map("${:,.2f}".format)

#reorder columns appropriately
col_order = ["Total Schools","Total Students", "Total Budget", "Ave Math Score", 
             "Ave Read Score", "% Pass Math", "% Pass Read", "% Overall Pass"]
dist_sum_df = dist_sum_df[col_order]

#print output
dist_sum_df

,Total Schools,Total Students,Total Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
0,15,"39,170","$24,649,428.00",79.0,81.9,75,86,65


In [356]:
#REQUIREMENT 2

#determine school types
per_school_types = sch_df.set_index(["school_name"])["type"]
per_school_types

#use value_counts for categorical counting
stud_per_school = stu_df["school_name"].value_counts()

#create series for basic budget table and do calculation
budget_per_school = school_types = sch_df.set_index(["school_name"])["budget"]
per_school_capita = budget_per_school/stud_per_school

#grab mean math and read per school
school_math_mean = all_data.groupby(["school_name"]).mean()["math_score"]
school_read_mean = all_data.groupby(["school_name"]).mean()["reading_score"]

#per school pass math/read create DF and count per DF 
school_pass_math = all_data[(all_data["math_score"] > 69)]
school_pass_read = all_data[(all_data["reading_score"] > 69)]
school_pass_overall = all_data[(all_data["math_score"] > 69) & (all_data["reading_score"] > 69)]
school_pass_m_count = school_pass_math.groupby(["school_name"]).count()["student_name"]
school_pass_r_count = school_pass_read.groupby(["school_name"]).count()["student_name"]
school_pass_all_count = school_pass_overall.groupby(["school_name"]).count()["student_name"]

#calculate percent pass per class per school
math_pass_perc_school = school_pass_m_count / stud_per_school * 100
read_pass_perc_school = school_pass_r_count / stud_per_school * 100
overall_pass_perc_school = school_pass_all_count / stud_per_school * 100

#create  and format output DF
school_sum_df = pd.DataFrame({
    "School Type" : per_school_types,
    "Total Students" : stud_per_school,
    "Total School Budget" : budget_per_school,
    "Per Student Budget" : per_school_capita,
    "Ave Math Score" : school_math_mean,
    "Ave Read Score" : school_read_mean,
    "% Pass Math" : math_pass_perc_school,
    "% Pass Read" : read_pass_perc_school,
    "% Overall Pass" : overall_pass_perc_school})

school_sum_df["Total Students"] = school_sum_df["Total Students"].map("{:,}".format)
school_sum_df["Ave Math Score"] = school_sum_df["Ave Math Score"].map("{:.1f}".format)
school_sum_df["Ave Read Score"] = school_sum_df["Ave Read Score"].map("{:.1f}".format)
school_sum_df["% Pass Math"] = school_sum_df["% Pass Math"].map("{:.2f}".format)
school_sum_df["% Pass Read"] = school_sum_df["% Pass Read"].map("{:.2f}".format)
school_sum_df["% Overall Pass"] = school_sum_df["% Overall Pass"].map("{:.2f}".format)
school_sum_df["Total School Budget"] = school_sum_df["Total School Budget"].map("${:,.2f}".format)

#print output
school_sum_df

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Bailey High School,District,"4,976","$3,124,928.00",628.0,77.0,81.0,66.68,81.93,54.64
Cabrera High School,Charter,"1,858","$1,081,356.00",582.0,83.1,84.0,94.13,97.04,91.33
Figueroa High School,District,"2,949","$1,884,411.00",639.0,76.7,81.2,65.99,80.74,53.20
Ford High School,District,"2,739","$1,763,916.00",644.0,77.1,80.7,68.31,79.30,54.29
Griffin High School,Charter,"1,468","$917,500.00",625.0,83.4,83.8,93.39,97.14,90.60
Hernandez High School,District,"4,635","$3,022,020.00",652.0,77.3,80.9,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",581.0,83.8,83.8,92.51,96.25,89.23
Huang High School,District,"2,917","$1,910,635.00",655.0,76.6,81.2,65.68,81.32,53.51
Johnson High School,District,"4,761","$3,094,650.00",650.0,77.1,81.0,66.06,81.22,53.54
Pena High School,Charter,962,"$585,858.00",609.0,83.8,84.0,94.59,95.95,90.54


In [357]:
#REQUIREMENT 3.1 Top 5 schools per overall passing

top_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = False)

top_sch.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Cabrera High School,Charter,"1,858","$1,081,356.00",582.0,83.1,84.0,94.13,97.04,91.33
Thomas High School,Charter,"1,635","$1,043,130.00",638.0,83.4,83.8,93.27,97.31,90.95
Griffin High School,Charter,"1,468","$917,500.00",625.0,83.4,83.8,93.39,97.14,90.60
Wilson High School,Charter,"2,283","$1,319,574.00",578.0,83.3,84.0,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",609.0,83.8,84.0,94.59,95.95,90.54


In [358]:
#REQUIREMENT 3.2 Bottom 5 schools per overall passing

top_sch = school_sum_df.sort_values(["% Overall Pass"], ascending = True)

top_sch.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Ave Math Score,Ave Read Score,% Pass Math,% Pass Read,% Overall Pass
Rodriguez High School,District,"3,999","$2,547,363.00",637.0,76.8,80.7,66.37,80.22,52.99
Figueroa High School,District,"2,949","$1,884,411.00",639.0,76.7,81.2,65.99,80.74,53.20
Huang High School,District,"2,917","$1,910,635.00",655.0,76.6,81.2,65.68,81.32,53.51
Hernandez High School,District,"4,635","$3,022,020.00",652.0,77.3,80.9,66.75,80.86,53.53
Johnson High School,District,"4,761","$3,094,650.00",650.0,77.1,81.0,66.06,81.22,53.54
